In [5]:
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import array_to_latex

def initialize_qubits(amplitudes):
    """
    Creates a quantum circuit that initializes two qubits to a given normalized state.
    :param amplitudes: List of four complex numbers representing the desired state amplitudes.
    :return: QuantumCircuit object with initialized qubits.
    """
    assert len(amplitudes) == 4, "Input must have four amplitudes."
    assert np.isclose(np.sum(np.abs(amplitudes)**2), 1), "Input state must be normalized."
    
    a0, a1, a2, a3 = amplitudes
    
    # Compute angles based on given formulas
    theta1 = 2 * np.arccos(np.sqrt(a0**2 + a2**2))
    theta2 = 2 * np.arctan2(a3, a1)
    theta3 = 2 * np.arctan2(a2, a0)
    
    # Create quantum circuit with 2 qubits
    qc = QuantumCircuit(2)
    
    # Apply Ry rotation on qubit 0
    qc.ry(theta1, 0)
    
    # Controlled Ry rotation on qubit 1 (controlled by qubit 0)
    qc.cry(theta2, 0, 1)
    
    # Apply X gate to qubit 0 (negation)
    qc.x(0)
    
    # Another controlled Ry rotation
    qc.cry(theta3, 0, 1)
    
    # Apply X gate again to qubit 0 to revert
    qc.x(0)
    
    return qc

def check_initialization(qc):
    """
    Uses the statevector simulator to check the resulting quantum state.
    """
    simulator = Aer.get_backend('statevector_simulator')
    transpiled_qc = transpile(qc, simulator)
    result = simulator.run(transpiled_qc).result()
    statevector = result.get_statevector()
    
    print("Resulting statevector:")
    display(array_to_latex(statevector, prefix="|ψ⟩ = "))
    return statevector

# Example test
amplitudes = np.sqrt(np.array([1/2, 1/4, 1/6, 1/12]))
qc = initialize_qubits(amplitudes)
qc.draw('mpl')

# Check initialization
statevector = check_initialization(qc)

print(statevector)


Resulting statevector:


<IPython.core.display.Latex object>

Statevector([0.70710678+0.j, 0.5       +0.j, 0.40824829+0.j,
             0.28867513+0.j],
            dims=(2, 2))
